Libraries and refrence link

In [ ]:
!pip install PyPDF2
!pip install docx2txt
!pip install -U sentence-transformers
!pip install sentence-transformers transformers
!pip install pytorch-pretrained-bert
!python -m spacy C:\Users\developer11\Downloads\AI Model\poc3 RH en_core_web_lg en
!pip install spacy
!python3 -m spacy download en_core_web_sm
!python -m pip install --upgrade pip
!pip install language_tool_python
# https://medium.com/featurepreneur/question-generator-d21265c0648f

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 218 kB 2.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3980 sha256=d3fa553ff88d0c35bbdde4af2cfe1b13ab7537a40bb356f5ed7ce718825f1b3a
  Stored in directory: /root/.cache/pip/wheels/b7/20/b2/473e3aea9a0c0d3e7b2f7bd81d06d0794fec12752733d1f3a8
Successfully built docx2txt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.8 MB/s 
     |████████████████████████████████| 4.9 MB 4.3 MB/s 
     |████████████████████████████████| 1.3 MB 50.0 MB/s 
     |████████████████████████████████| 120 kB 46.3 MB/s 
     |████████████████████████████████| 6.6 MB 48.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers

## Import Libries

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import json
import requests
import docx2txt
import urllib.request
import requests, PyPDF2
import pandas as pd
import numpy as np
import string
import re
import time

from nltk import sent_tokenize
from sentence_transformers import SentenceTransformer
from transformers import T5ForConditionalGeneration, T5Tokenizer, BertTokenizer, BertModel, AutoTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import spacy
from transformers import BertTokenizer, BertModel
from warnings import filterwarnings as filt
from warnings import filterwarnings as filt
filt('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
start = time.time()

## **Import File**

In [ ]:
request=requests.get("https://generate-questions.devbyopeneyes.com/api/getFileData/6321af017e09973e7309fea2")
resp= request.json()
file_path=(resp["data"]["file_path"])
_id=(resp["data"]["_id"])
number_of_question=(resp["data"]["number_of_question"])
file_type=(resp["data"]["file_type"])

## Read Document : txt, pdf, docx

In [ ]:
if file_type=="txt":
  
  response = urllib.request.urlopen(file_path)
  html = response.read()
  text=html.decode('utf8')

elif file_type=="pdf":
  url = file_path
  response = requests.get(url)
  my_raw_data = response.content

  with open("my_pdf.pdf", 'wb') as my_data:
      my_data.write(my_raw_data)

  open_pdf_file = open("my_pdf.pdf", 'rb')
  read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
  if read_pdf.isEncrypted:
      read_pdf.decrypt("")
      text = read_pdf.getPage(0).extractText()

  else:
      text = read_pdf.getPage(0).extractText()
        
elif file_type=="docx":
  url = file_path
  response = requests.get(url)
  my_raw_data = response.content

  with open("my_doc.txt", "wb") as text_file:
    text_file.write(my_raw_data)

  open_docx_file = open("my_doc.txt", 'rb')

  text = docx2txt.process(open_docx_file)

else:
  print("Invalid File Type")


## File Validation : Empty file, File can contain special character only

In [ ]:
if len(text)==0:
  print("file is empty.")
else:
  if not re.match(r'!@#$%^&"*()-+?_=,<>/\'', text) :
    print("Valid file")
  elif text=="'":
    print("invalidd")
  else:
    print('Invalid file')

Valid file


## **Data Cleaning And Data Preprossing**

**Split text file into sentences**

In [ ]:
sentences = sent_tokenize(text)
text_to_sentence= np.array(sentences)

**Split sentences into words by white space**

In [ ]:
sentence_to_words=[]
for i in text_to_sentence[0:]:
    sentence_to_words.extend(i.split()) 

**Remove Unwanted Words** 

In [ ]:
tokens = [ w for w in sentence_to_words if w[0]!='[' and w[-1]!= ']' ]

**Remove punctuation from each word(except full stop(.))**

In [ ]:
remove = string.punctuation
remove = remove.replace(".", "")
pattern = r"[{}]".format(re.escape(remove))
table = str.maketrans('', '', pattern)
stripped = [w.translate(table) for w in tokens]

**Join Sentence**

In [ ]:
words_to_sentense=' '.join(stripped)

## **Generate_WH_questions_using_T5_transformer**

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained("bert-base-uncased")
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
nlp = spacy.load("en_core_web_sm")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

## Questions generation

In [ ]:
def get_question(sentence, answer):

  mdl = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
  tknizer = AutoTokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')

  text = "context: {} answer: {}".format(sentence,answer)
  max_len = 256
  encoding = tknizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = mdl.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=300)


  dec = [tknizer.decode(ids,skip_special_tokens=True) for ids in outs]


  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question

In [ ]:
def get_embedding(doc):

  bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  bert_model = BertModel.from_pretrained("bert-base-uncased")
  
  # txt = '[CLS] ' + doc + ' [SEP]'
  tokens = bert_tokenizer.tokenize(txt)
  token_idx = bert_tokenizer.convert_tokens_to_ids(tokens)
  segment_ids = [1] * len(tokens)

  torch_token = torch.tensor([token_idx])
  torch_segment = torch.tensor([segment_ids])

  return bert_model(torch_token, torch_segment)[-1].detach().numpy()

def get_pos(context):
  doc = nlp(context)
  docs = [d.pos_ for d in doc]
  return docs, context.split()

def get_sent(context):
  doc = nlp(context)
  return list(doc.sents)

def get_vector(doc):
  stop_words = "english"
  n_gram_range = (1,1)
  df = CountVectorizer(ngram_range = n_gram_range, stop_words = stop_words).fit([doc])
  return df.get_feature_names()


def get_key_words(context, module_type = 't'):
  keywords = []
  top_n = 5
  for txt in get_sent(context):
    keywd = get_vector(str(txt))
    print(f'vectors : {keywd}')
    if module_type == 't':
      doc_embedding = get_embedding(str(txt))
      keywd_embedding = get_embedding(' '.join(keywd))
    else:
      doc_embedding = model.encode([str(txt)])
      keywd_embedding = model.encode(keywd)
    
    distances = cosine_similarity(doc_embedding, keywd_embedding)
    print(distances)
    keywords += [(keywd[index], str(txt)) for index in distances.argsort()[0][-top_n:]]

  return keywords

## Stored a Questions, Context, Answer, Score And test Duplicate data

In [ ]:
txt = words_to_sentense
questions_arr=[]

for answer, context in get_key_words(txt, 'st'):
    questions_arr.append({"question":get_question(context, answer),"context":context,"answer":answer, 'rank':None})


### Remove Duplications

In [ ]:
new_questions = []
for i in questions_arr:
  if i not in new_questions:
    new_questions.append(i)

## Get Questiosn from new_questions for the score

In [ ]:
scoring = []
for i in new_questions:
    questions_list = i.get("question")
    scoring.append(questions_list)   

## Eevaluation of Score

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
context = words_to_sentense
question_score = []
for i in scoring:
      final_score_list = question_answerer(question=i, context=context)
      final_score = final_score_list.get("score")
      final_score ="%.2f" % round(final_score*100, 2)
      question_score.append(final_score)

## Replace the score value

In [ ]:
for option_index, option in enumerate(new_questions):
    option["rank"] = question_score[option_index]

In [ ]:
final_generated_questions = sorted(new_questions, key=lambda i: i['rank'],reverse=True)

## Question seperation based on user input

In [ ]:
user_input=int(number_of_question)
if user_input > len(final_generated_questions) or user_input == 0: 
  print("please give value between 1 to {}".format(len(final_generated_questions)))
else:
    middle_index = user_input
    user_required_questions = final_generated_questions[:middle_index]
    other_questions = final_generated_questions[middle_index:]

    
    for question_index, question in enumerate(user_required_questions):
      question["question_id"]= question_index +1

    for question_index, question in enumerate(other_questions):
      question["question_id"]= question_index +1

In [ ]:
end = time.time()

In [ ]:
from datetime import datetime

dt1 = datetime.fromtimestamp(start)
dt2 = datetime.fromtimestamp(end)
total_time = dt2 - dt1

## **Final Output , User Validation & Post Generated Questions**

In [ ]:
url="https://generate-questions.devbyopeneyes.com/api/GenerateQuestions" 
headers = {'Content-Type':'application/json','Accept':'application/json'}
post_array = {  "id" : _id,
                "questions" : user_required_questions,
                "other_questions" : other_questions,
                "upload_process_time": str(total_time)
    }
status = requests.post(url,headers=headers,data=json.dumps(post_array))

In [ ]:
print(post_array)